In [92]:
import random
import optuna
import mlflow
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [93]:
def read_data(filename: str) -> pd.DataFrame:
    """Reads data from a pickle file, removes unused columns.

    Args:
        filename (str): Path to the file.
    
    Returns:
        pd.DataFrame: Pandas DataFrame object.
    """
    
    # TODO Verify data format and pre-defined schema.

    cols_to_drop = ['subba-name', 'parent', 'parent-name', 'value-units']
    dtypes = {'period': 'datetime64',
              'subba': 'category',
              'timezone': 'category',
              'value': 'uint64'}

    # Load the data, remove extra columns, and adjust data types.
    df = pd.read_pickle(filename).drop(columns=cols_to_drop)
    df = df.drop_duplicates().dropna().reset_index(drop=True)
    df = df.astype(dtypes)
    
    # We focus solely on New York City data for our task.
    df = df[df['subba'] == 'ZONJ']
    return df


# print('df shape:', df.shape)
# print(df.head(1))
# print(df.tail(1))

In [94]:
def filter_by_iqr(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filter DataFrame using the Interquartile Range (IQR) method.

    Args:
        df (pd.DataFrame): DataFrame containing the "value" column.

    Returns:
        pd.DataFrame: Filtered DataFrame with outliers removed.
    """
    # Compute the IQR for the "value" column
    Q1 = df['value'].quantile(0.25)
    Q3 = df['value'].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the acceptable range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove rows where the "value" column is outside the bounds.
    filt = (df['value'] >= lower_bound) & (df['value'] <= upper_bound)
    df_filtered = df[filt]

    return df_filtered

def plot_data(df: pd.DataFrame) -> None:
    """
    Plot the data from a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with 'period' and 'value' columns.
    """
    # Sort the DataFrame by "period" and plot the data
    x = 'period'
    y = 'value'
    
    df.sort_values(by='period').plot(x=x, y=y, figsize=(10, 5))

    # Set the axis labels and title
    plt.xlabel('Period')
    plt.ylabel('Value')
    plt.title('Value over Time')

    # Display the plot
    plt.show()

# plot_data(df_no_outliers)

In [95]:
def extract_features(
        df: pd.DataFrame,
        window_size=7
) -> pd.DataFrame:
    """
    Extract date-related and rolling statistics features from DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with 'period' and 'value' columns.

    Returns:
        pd.DataFrame: DataFrame with added date features and rolling stats.
    """
    df = df.copy()
    
    # Separate date features
    df['year'] = df['period'].dt.year.astype('uint16')
    df['month'] = df['period'].dt.month.astype('uint16')
    df['day'] = df['period'].dt.day.astype('uint16')

    # Extract the day of the week
    df['day_of_week'] = df['period'].dt.day_name().astype('category')

    # Extract quarterly and weekly information to capture seasonality
    df['quarter'] = df['period'].dt.quarter.astype('uint8')
    df['week_of_year'] = df['period'].dt.isocalendar().week.astype('uint16')

    # Mark the weekends
    df['is_weekend'] = (df['period'].dt.weekday >= 5).astype('uint8')
    
    # Create rolling mean feature
    df['rolling_mean'] = df['value'].rolling(window=window_size).mean()
    
    # Create rolling standard deviation feature
    df['rolling_std'] = df['value'].rolling(window=window_size).std()
    
    # Remove NaNs introduced by the rolling mean and std
    df = df.dropna().copy().reset_index(drop=True)
    print('df after dropna',df)

    # Create a random feature as a threshold for later feature filtering
    random_feature = [random.random() for _ in range(df.shape[0])]
    df['random_feature'] = random_feature
    
    return df

In [96]:
def transform_to_supervised(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transform DataFrame into a supervised learning format.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: DataFrame with 'lag' column and 'period' dropped.
    """
    
    # Create 'lag' column by shifting 'value' column
    df.loc[:, 'lag'] = df['value'].shift()

    # Remove rows with missing values (NaN) due to the shift
    df.dropna(inplace=True)

    # Convert 'lag' column to unsigned 64-bit integer type
    df.lag = df.lag.astype('uint64')

    # Drop the 'period' column from the DataFrame
    df = df.drop(columns=['period'])

    return df

def encode_categorical(df: pd.DataFrame, ohe=None, fit=True) -> pd.DataFrame:
    """
    Encode categorical columns as dummy variables.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: DataFrame with categorical columns encoded as dummy variables.
    """ 
    df = df.reset_index(drop=True)
    
    if ohe == None:
        ohe = OneHotEncoder(sparse_output=False)

    # List of columns to encode as dummy variables
    feat_to_enc = df.select_dtypes('category')

    # Get dummy variables for the specified columns
    if fit == True:
        dummies = pd.DataFrame(ohe.fit_transform(feat_to_enc))
    else:
        dummies = pd.DataFrame(ohe.transform(feat_to_enc))
    dummies.columns = ohe.get_feature_names_out()
    
    # Concatenate the dummy variables with the original DataFrame
    df = pd.concat((df.drop(columns=feat_to_enc.columns), dummies), axis=1)
    return df, ohe

In [97]:
def train(
    model: XGBRegressor, 
    X: pd.DataFrame, 
    y: pd.Series,
    n_splits: int
) -> tuple:
    """
    Train a model using TimeSeriesSplit cross-validation.

    Args:
        model (XGBRegressor): XGBoost regressor using scikit-learn API.
        X (pd.DataFrame): Features DataFrame.
        y (pd.Series): Target values Series.

    Returns:
        tuple: Contains average Mean Absolute Error on the training 
              set (float) and test set (float) respectively.
    """
    # Lists to store mean absolute errors for training and test sets
    mae_train_hist = []
    mae_test_hist = []
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    # Split the data using TimeSeriesSplit for cross-validation
    for train_index, test_index in tscv.split(X):
        # Split the data into training and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        
        # Predict on training and test sets
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate mean absolute error for training and test sets
        mae_train = mean_absolute_error(y_train, y_pred_train)
        mae_test = mean_absolute_error(y_test, y_pred_test)
        
        # Append the errors to their respective histories
        mae_train_hist.append(mae_train)
        mae_test_hist.append(mae_test)

    # Calculate average mean absolute error for training and test sets
    mae_train_avg = sum(mae_train_hist) / len(mae_train_hist)
    mae_test_avg = sum(mae_test_hist) / len(mae_test_hist)
    
    return (mae_train_avg, mae_test_avg)


In [98]:
def remove_redundant_features(
        model: XGBRegressor, 
        X: pd.DataFrame
) -> pd.DataFrame:
    """
    Remove features less important than a reference 'random_feature'.
    
    Args:
        model (XGBRegressor): XGBoost regressor using scikit-learn API.
        X (pd.DataFrame): Features DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame with redundant features removed.
    """
    # Get feature importances and feature names
    importances = model.feature_importances_
    features = X.columns.tolist()
    
    # Create a DataFrame for feature importances
    feat_imps = pd.DataFrame({
        'feature': features,
        'importance': importances
    })
    
    # Sort features by importance
    sorted_feat_imps = feat_imps.sort_values(by='importance', ascending=False)
    sorted_feat_imps.reset_index(drop=True, inplace=True)
    
    # Find the importance threshold of 'random_feature'
    random_feature_row = sorted_feat_imps[
            sorted_feat_imps['feature'] == 'random_feature']
    thresh = random_feature_row['importance'].iloc[0]
    
    # Identify columns to drop (less important than 'random_feature')
    redundant_features = sorted_feat_imps[
            sorted_feat_imps['importance'] < thresh]
    to_drop = redundant_features['feature'].tolist()
    to_drop.append('random_feature')
    
    # Drop redundant columns
    X_new = X.drop(columns=to_drop)
    
    return (X_new, to_drop)

In [99]:
def tune_hyperparameters(X, y, n_trials=100, n_splits=5):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 2, 1000),
            'max_depth': trial.suggest_int('max_depth', 1, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'alpha': trial.suggest_float("alpha", 1e-5, 10, log=True),  
            'gamma': trial.suggest_float("gamma", 1e-5, 5, log=True),
            'lambda': trial.suggest_float('lambda', 1e-5, 10.0, log=True),
            'early_stopping_rounds': 50
        }

        model = XGBRegressor(**params)
        mae_train_avg, mae_test_avg = train(model, X, y, n_splits)
        return mae_test_avg

    study = optuna.create_study(direction='minimize')

    # Optimize the study, the objective function is passed in as the first argument
    study.optimize(objective, n_trials=n_trials)

    # Results
    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial
    
    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')
    
    return trial.params

In [102]:
def main(n_trials=1) -> XGBRegressor:
    # Load the data.
    filename = 'data/2018-01-01-2022-01-01.bin'
    df = read_data(filename)

    # Clean it, extract date features and running statistics.
    df_no_outliers = filter_by_iqr(df)
    df_newfeat= extract_features(df_no_outliers)
    
    # Preprocess the dataset for model input.
    df_transformed = transform_to_supervised(df_newfeat)
    df_encoded, ohe = encode_categorical(df_transformed)
    
    # Separate features and target.
    X = df_encoded.drop(columns=['value'])
    y = df_encoded.value

    model = XGBRegressor(objective='reg:squarederror', n_estimators=100)
    n_splits = 9
    mae_train_avg, mae_test_avg = train(model, X, y, n_splits=n_splits)
    print(f'Avg MAE:\nTrain: {mae_train_avg}\nTest: {mae_test_avg}')
    
    X_new, features_to_drop = remove_redundant_features(model, X)
    
    best_params = tune_hyperparameters(X_new, y, n_trials=n_trials)
    model = XGBRegressor(**best_params)
    train(model, X_new, y, n_splits=n_splits)

    return model, ohe, features_to_drop

In [103]:
model, ohe, features_to_drop = main()

df after dropna          period subba  timezone   value  year  month  day day_of_week  \
0    2021-12-31  ZONJ   Arizona  117623  2021     12   31      Friday   
1    2021-12-31  ZONJ  Mountain  117623  2021     12   31      Friday   
2    2021-12-31  ZONJ   Central  117593  2021     12   31      Friday   
3    2021-12-31  ZONJ   Eastern  117546  2021     12   31      Friday   
4    2021-12-30  ZONJ   Eastern  123116  2021     12   30    Thursday   
...         ...   ...       ...     ...   ...    ...  ...         ...   
6034 2018-06-26  ZONJ   Pacific  154798  2018      6   26     Tuesday   
6035 2018-06-26  ZONJ  Mountain  154792  2018      6   26     Tuesday   
6036 2018-06-26  ZONJ   Eastern  154928  2018      6   26     Tuesday   
6037 2018-06-26  ZONJ   Central  154836  2018      6   26     Tuesday   
6038 2018-06-26  ZONJ   Arizona  154798  2018      6   26     Tuesday   

      quarter  week_of_year  is_weekend   rolling_mean  rolling_std  
0           4            52          

[I 2023-08-07 13:22:13,793] A new study created in memory with name: no-name-0a8420b4-2245-448e-84a5-095edad7a7b5


Avg MAE:
Train: 398.1347730885192
Test: 3231.9671017021374


[I 2023-08-07 13:22:14,421] Trial 0 finished with value: 3506.428125 and parameters: {'n_estimators': 44, 'max_depth': 15, 'learning_rate': 0.14756516106505638, 'subsample': 0.7078301349974585, 'colsample_bytree': 0.5321121017741631, 'min_child_weight': 6, 'alpha': 4.3860756369466297e-05, 'gamma': 0.009759538884952212, 'lambda': 6.397801747345744e-05}. Best is trial 0 with value: 3506.428125.


Number of finished trials:  1
Best trial:
Value:  3506.428125
Params: 
    n_estimators: 44
    max_depth: 15
    learning_rate: 0.14756516106505638
    subsample: 0.7078301349974585
    colsample_bytree: 0.5321121017741631
    min_child_weight: 6
    alpha: 4.3860756369466297e-05
    gamma: 0.009759538884952212
    lambda: 6.397801747345744e-05


In [ ]:
def prepare_for_inference(df: pd.DataFrame):
    
    return df

In [90]:
# Test inference on unseen data
fname = 'testdata/2022-01-02-2022-01-02.bin'
# fname = 'data/2018-01-01-2022-01-01.bin'
test_df = read_data(fname).iloc[0,:].to_frame().T.reset_index(drop=True)
# print(test_df)
print('After loading:', test_df.shape)
df_newfeat, _ = extract_features(
    test_df, 
    window_size=1, 
    inference=True, 
    rolling_stats=saved_rolling_stats
)
print(df_newfeat)
print('After feature extraction:', df_newfeat.shape)
df_transformed = transform_to_supervised(df_newfeat)
print('After transform:', df_transformed.shape)
print(df_transformed.columns.tolist())
df_encoded, _ = encode_categorical(df_transformed, ohe=ohe, fit=False)
print('After encoding:', df_encoded.shape)

to_drop = ['value'] + features_to_drop
X = df_encoded.drop(columns=to_drop)
print('After separating features from target:',X.shape)
y_true = df_encoded.value

y_pred = model.predict(X)

mean_absolute_error(y_true, y_pred)
print('y_true:', y_true, '\ny_pred:', y_pred)

After loading: (1, 4)
df after dropna       period subba timezone   value  year  month  day day_of_week  quarter  \
0 2022-01-02  ZONJ  Pacific  112878  2022      1    2      Sunday        1   

   week_of_year  is_weekend   rolling_mean  rolling_std  
0            52           1  154114.571429  1225.669729  
      period subba timezone   value  year  month  day day_of_week  quarter  \
0 2022-01-02  ZONJ  Pacific  112878  2022      1    2      Sunday        1   

   week_of_year  is_weekend   rolling_mean  rolling_std  random_feature  
0            52           1  154114.571429  1225.669729        0.888663  
After feature extraction: (1, 14)
After transform: (0, 14)
['subba', 'timezone', 'value', 'year', 'month', 'day', 'day_of_week', 'quarter', 'week_of_year', 'is_weekend', 'rolling_mean', 'rolling_std', 'random_feature', 'lag']


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- subba
- timezone
